In [3]:
%matplotlib inline
import sys
#print(sys.path)
sys.path.append("/Users/raroito/PycharmProjects/facial_expression_recognition/src/")

from utils import ModelImporter, Preprocessing
import torch
import pandas as pd
import numpy as np

# Analize Test Data Results

In [8]:
n_classes = 7
n_epochs = 100
learning_rate = 0.0001
batch_size = 32

model_name = f'cnn_simple_{learning_rate}_{batch_size}_{n_epochs}_{n_classes}'

In [5]:
pre = Preprocessing('fer2013')
pre.load_data(filename='test_public_norm.csv', name='test')

X_df = pre.get(name='test').drop(columns=['emotion'])
y_df = pre.get(name='test')['emotion']

In [9]:
dtype = torch.float
device = torch.device("cpu")

m_importer = ModelImporter('fer2013')
model = m_importer.load_nn_model(model_name)

load model CnnSimple(
  (drop_hidden): Dropout(p=0.5)
  (drop_visible): Dropout(p=0.2)
  (conv1): Conv2d(1, 15, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batchnorm1): BatchNorm2d(15, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=8640, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=7, bias=True)
)


In [10]:
X_test = model.reshape_data(torch.tensor(X_df.values, device=device, dtype=dtype))
y_test = torch.tensor(y_df.values, device=device, dtype=torch.long)

In [11]:
y_pred_prob = model(X_test)

In [12]:
y_pred = y_pred_prob.argmax(1)

In [13]:
y_pred_prob

tensor([[ 1.1495e+00, -8.3653e+00,  9.9272e-01,  ...,  2.0777e-01,
         -2.5419e+00, -6.3208e-02],
        [ 1.0451e+00, -3.0641e+00,  4.3036e-01,  ...,  1.5992e-01,
         -1.4975e+00, -2.4168e-01],
        [ 1.2575e+00, -1.1175e+00,  1.2134e-01,  ...,  2.3720e-01,
         -2.0039e+00,  2.3446e-01],
        ...,
        [ 5.9072e-01, -7.7584e+00,  1.2306e+00,  ..., -5.2755e-04,
          1.8469e-01,  1.3503e+00],
        [ 1.3151e+00, -4.6900e+00, -1.3014e-01,  ...,  1.0648e+00,
         -6.1970e+00,  9.9498e-01],
        [-1.7100e+00, -1.0138e+01, -3.2636e-01,  ..., -9.4807e-01,
         -9.0829e-01,  2.1861e+00]], grad_fn=<AddmmBackward>)

In [14]:
y_pred

tensor([0, 0, 0,  ..., 6, 0, 6])

In [21]:
accuracy_soft = (y_pred == y_test).float().mean()
accuracy_soft

tensor(0.4681)

In [15]:
results = pd.DataFrame()

In [16]:
results['y_test'] = y_test
results['y_pred'] = y_pred
results['right'] = (y_pred == y_test).float()
results['ones'] = 1

In [17]:
results.tail()

,y_test,y_pred,right,ones
3584,4,0,0.0,1
3585,3,3,1.0,1
3586,4,6,0.0,1
3587,4,0,0.0,1
3588,4,6,0.0,1


In [18]:
report = results.groupby('y_test').sum()
report

,y_pred,right,ones
y_test,,,
0,1168,164.0,467
1,112,8.0,56
2,1607,145.0,496
3,2847,628.0,895
4,2190,230.0,653
5,1747,271.0,415
6,2411,234.0,607


In [19]:
report['%'] = report['right']/report['ones']
report

,y_pred,right,ones,%
y_test,,,,
0,1168,164.0,467,0.351178
1,112,8.0,56,0.142857
2,1607,145.0,496,0.292339
3,2847,628.0,895,0.701676
4,2190,230.0,653,0.352221
5,1747,271.0,415,0.653012
6,2411,234.0,607,0.385502


In [71]:
report['ones'].sum()

3589

In [58]:
#class 3 is somhow well recognized (also is the class with more samples) 
#whereas not a single sample of class 2 was properly detected

# Analize the Training Dataset

In [59]:
pre = Preprocessing('fer2013')
pre.load_data(filename='train_norm.csv', name='train')

X_df = pre.get(name='train').drop(columns=['emotion'])
y_df = pre.get(name='train')['emotion']

In [60]:
X_train= model.reshape_data(torch.tensor(X_df.values, device=device, dtype=dtype))
y_train = torch.tensor(y_df.values, device=device, dtype=torch.long)

In [61]:
y_pred_prob = model(X_train)

In [62]:
y_pred = y_pred_prob.argmax(1)

In [63]:
y_pred_prob

tensor([[9.4279e-01, 7.7489e-13, 7.2470e-02,  ..., 9.3389e-01, 3.4865e-04,
         2.0234e-01],
        [9.9909e-01, 1.4779e-13, 5.9159e-01,  ..., 9.3531e-01, 2.5739e-04,
         2.2301e-02],
        [2.1908e-03, 4.5221e-16, 9.6216e-03,  ..., 8.4227e-02, 4.5770e-01,
         3.3807e-01],
        ...,
        [4.7102e-01, 4.2740e-13, 5.6189e-01,  ..., 9.9913e-01, 2.0440e-04,
         5.5143e-01],
        [1.6083e-03, 5.7853e-12, 5.9794e-01,  ..., 3.9897e-03, 9.7383e-01,
         4.0441e-01],
        [3.3839e-01, 6.5697e-15, 6.9736e-01,  ..., 6.0949e-02, 2.2432e-01,
         2.9055e-01]], grad_fn=<SigmoidBackward>)

In [64]:
y_pred

tensor([0, 0, 5,  ..., 4, 5, 2])

In [65]:
y_train

tensor([0, 0, 2,  ..., 4, 0, 4])

In [66]:
results_train['y_train'] = y_train
results_train['y_pred'] = y_pred
results_train['right'] = (y_pred == y_train).float()
results_train['ones'] = 1

In [67]:
report_train = results_train.groupby('y_train').sum()
report_train

,y_pred,right,ones
y_train,,,
0,10777,1336.0,3995
1,1302,0.0,436
2,13513,839.0,4097
3,22852,5273.0,7215
4,17240,1232.0,4830
5,13354,1793.0,3171
6,22026,2568.0,4965


In [68]:
report_train['%'] = report_train['right']/report_train['ones']
report_train

,y_pred,right,ones,%
y_train,,,,
0,10777,1336.0,3995,0.334418
1,1302,0.0,436,0.000000
2,13513,839.0,4097,0.204784
3,22852,5273.0,7215,0.730839
4,17240,1232.0,4830,0.255072
5,13354,1793.0,3171,0.565437
6,22026,2568.0,4965,0.517221


In [ ]:
#similar results with the train data as with the 